Syntax natürlicher Sprachen, WS 2023/24

# 06 - Aufgabenblatt (Lösung)

In [1]:
import nltk
from nltk import Tree
from nltk import DependencyGraph
from spacy import displacy
import spacy
from itertools import chain

In [2]:
def _tree_labeled(self, i):
    node = self.get_by_address(i)
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word + '(' + rel + ')', [self._tree_labeled(dep) for dep in deps]) if deps else word + '(' + rel + ')'

def tree_labeled(self):
    node = self.root
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled

def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split(); line_list.pop(0); line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))
    return "\n".join([i for i in sent_list[0:]])

def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())
    deps = [x for x in deps if x]
    ex, word_list, arc_list = [], [], []
    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1; head = int(dep[2]); label = dep[3]
        if head>line:
            start = index; end = head-1; direction = "left"
        else:
            start = head-1; end = index; direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})
    ex.append({"words": word_list,"arcs": arc_list})
    return ex

def show_dep_trees(sent_nr,style=1):
    sent = transform_nr_conll(sent_nr)
    dg = DependencyGraph(sent)
    if style == 0 or style == 2:
        tree_labeled = dg.tree_labeled()
        tree_labeled.pretty_print(unicodelines=True)   
    if style == 1 or style == 2:
        ex = displacy_dep_input(sent)
        html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

## Aufgabe 1 - Dependenzanalyse

### a) Analysieren Sie die Dependenzbeziehungen des folgenden Satzes im UD-Schema:
*Ich kenne einen älteren Bruder von zwei sehr lebhaften Mädchen*


In [3]:
sent_nr = """
1 Ich 2 nsubj 
2 kenne 0 ROOT
3 einen 5 det
4 älteren 5 amod 
5 Bruder 2 obj
6 von 10 case
7 zwei 10 nummod
8 sehr 9 advmod
9 lebhaften 10 amod  
10 Mädchen 5 nmod
"""

show_dep_trees(sent_nr)

### b) Analysieren Sie die Dependenzbeziehungen des folgenden Satzes im UD-Schema:
*Fährt ein elektrisches Auto von Tesla schnell ?*


In [4]:
sent_nr = """
1 fährt 0 ROOT
2 ein 4 det
3 elektrisches 4 amod
4 Auto 1 nsubj
5 von 6 case
6 Tesla 4 nmod
7 schnell 1 advmod
8 ? 1 punct
"""

show_dep_trees(sent_nr)

## Aufgabe 2 - Dependenzanalyse und Erweiterung

### a) Analysieren Sie die Dependenzbeziehungen dieses einfachen ditransitiven Satzes im UD-Schema:
*Gestern übergab ein Briefträger ihr den Brief*

In [5]:
sent_nr = """
1 Gestern 2 advmod
2 übergab 0 ROOT
3 ein 4 det
4 Briefträger 2 nsubj
5 ihr 2 iobj
6 den 7 det
7 Brief 2 obj
"""

show_dep_trees(sent_nr)

In [6]:
spacy.explain('iobj')

'indirect object'

In [7]:
spacy.explain('advmod')

'adverbial modifier'

### b) Ersetzen Sie das durch ein Adverb realisierte Adverbial durch ein Kasus-Adverbial:


In [8]:
sent_nr = """
1 Dieser 2 det
2 Tage 3 obl
3 übergab 0 ROOT
4 ein 5 det
5 Briefträger 3 nsubj
6 ihr 3 iobj
7 den 8 det
8 Brief 3 obj
"""

show_dep_trees(sent_nr)

### c) Fügen Sie zu dem Ausgangssatz ein Genitiv-Attribut hinzu:


In [9]:
sent_nr = """
1 Gestern 2 advmod
2 übergab 0 ROOT
3 ein 4 det
4 Briefträger 2 nsubj
5 ihr 2 iobj
6 den 7 det
7 Brief 2 obj
8 ihrer 9 det
9 Freundin 7 nmod
"""

show_dep_trees(sent_nr)

## Aufgabe 3 - Satzglied-Funktion


### a) Objekt vs Adverbial

#### Gegeben sei folgender Satz, dessen drittes Satzglied den *geschehens*-Test nicht besteht. 


In [10]:
sentence = ["er", "isst", "einen Apfel"]

sentence[0] + " " + sentence[1] + ", und das geschieht " + sentence[2]

'er isst, und das geschieht einen Apfel'

#### Für welchen Typ von Dependent spricht dies (Angabe/Adjunkt oder Ergänzung/Komplement)?


In [ ]:
#Ergänzung/Komplement


#### Welche syntaktische Funktion hat das dritte Satzglied hier?  (Objekt oder Adverbial?) Testen Sie über Passivierung!

In [ ]:
# Objekt (Passivierung möglich)
# Passivierung: ein Apfel wird gegessen

### b) Präpositionalobjekt vs Adverbial

#### Gegeben sei folgender Satz, dessen drittes Satzglied den *geschehens*-Test besteht (der Satz bleibt wohlgeformt). 


In [11]:
sentence = ["er", "wartet", "auf dem Berggipfel"]

sentence[0] + " " + sentence[1] + ", und das geschieht " + sentence[2]

'er wartet, und das geschieht auf dem Berggipfel'

#### Für welchen Typ von Dependent spricht dies (Angabe/Adjunkt oder Ergänzung/Komplement)?


In [ ]:
#Angabe/Adjunkt


#### Welche syntaktische Funktion hat das dritte Satzglied hier?  (Objekt oder Adverbial?) Testen Sie über Erfragbarkeit mit Pronominaladverb!

In [ ]:
#Adverbial (Test nicht möglich)
# Test Erfragbarkeit:  * worauf wartet er?  > auf dem Berg.

### c) Analysieren Sie die Dependenzbeziehungen des vorherigen Satzes im UD-Schema.

#### Beachten Sie ggf. die Analysekonvention im UD-Schema für Präpositionalobjekte.


In [12]:
sent_nr = """
1 er 2 nsubj
2 wartet 0 ROOT
3 auf 5 case
4 dem 5 det
5 Berggipfel 2 obl
"""

show_dep_trees(sent_nr)